In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%capture
%run "2. Definir variables.ipynb"

In [ ]:
def Analizar_Variable_Dependiente(Variable_Y, Nombre_Variable):

    """
    Analiza exhaustivamente las características estadísticas de una variable 
    dependiente para determinar el modelo de regresión más apropiado.
    
    Esta función examina la distribución, características clave y propiedades
    estadísticas de la variable dependiente, proporcionando recomendaciones
    específicas sobre qué tipo de modelo de regresión utilizar basándose en
    los patrones observados en los datos.
    
    Parámetros:
    -----------
    Variable_Y : pandas.Series
        Serie que contiene los valores de la variable dependiente a analizar.
        Debe contener datos numéricos.
    
    Nombre_Variable : str
        Nombre descriptivo de la variable dependiente que se utilizará
        en los títulos y etiquetas del análisis.
    
    Retorna:
    --------
    dict
        Diccionario con las características booleanas detectadas:
        - 'Es_Ordinal': bool - True si la variable tiene características ordinales
        - 'Tiene_Muchos_Ceros': bool - True si más del 50% son valores cero
        - 'Es_Simetrica': bool - True si la distribución es simétrica (|skew| < 0.5)
        - 'Rango_Limitado': bool - True si el rango total es ≤ 10
    
    Ejemplo de uso:
    ---------------
    >>> import pandas as pd
    >>> datos = pd.Series([0, 0, 1, 2, 3, 0, 1, 0, 2])
    >>> resultado = Analizar_Variable_Dependiente(datos, "Número_Compras")
    >>> print(resultado['Tiene_Muchos_Ceros'])  # True
    
    Notas:
    ------
    - La función imprime un reporte completo en consola con estadísticas
      descriptivas y recomendaciones de modelos.
    - Las recomendaciones incluyen: regresión ordinal, modelos zero-inflated,
      regresión gamma/log-normal, logística y Poisson según corresponda.
    
    """
    
    print(f"ANÁLISIS DE {Nombre_Variable}:")
    print("="*50)
    
    # Distribución de valores.
    print("Distribución de valores:")
    Distribucion = Variable_Y.value_counts().sort_index()
    Porcentajes = Variable_Y.value_counts(normalize=True).sort_index() * 100
    
    for Valor in sorted(Variable_Y.unique()):
        Frecuencia = Distribucion[Valor]
        Porcentaje = Porcentajes[Valor]
        print(f"  Valor {Valor:2.0f}: {Frecuencia:4d} obs ({Porcentaje:5.1f}%)")
    
    # Características clave.
    Es_Ordinal = len(Variable_Y.unique()) <= 10 and Variable_Y.min() >= -10 and Variable_Y.max() <= 10
    Tiene_Muchos_Ceros = (Variable_Y == 0).mean() > 0.5
    Es_Simetrica = abs(Variable_Y.skew()) < 0.5
    Rango_Limitado = Variable_Y.max() - Variable_Y.min() <= 10
    
    print(f"\nCaracterísticas:")
    print(f"  Valores únicos: {Variable_Y.nunique()}")
    print(f"  Rango: [{Variable_Y.min()}, {Variable_Y.max()}]")
    print(f"  Media: {Variable_Y.mean():.3f}")
    print(f"  Mediana: {Variable_Y.median():.3f}")
    print(f"  Desviación estándar: {Variable_Y.std():.3f}")
    print(f"  Asimetría: {Variable_Y.skew():.3f}")
    print(f"  Exceso de ceros: {(Variable_Y == 0).mean()*100:.1f}%")
    
    # Recomendación de modelo.
    print(f"\nRECOMENDACIÓN DE MODELO:")
    
    if Es_Ordinal and Rango_Limitado:
        print("  ✅ REGRESIÓN ORDINAL - Variable con categorías ordenadas")
        
    if Tiene_Muchos_Ceros:
        print("  ✅ MODELO ZERO-INFLATED - Exceso de valores cero")
        
    if not Es_Simetrica and Variable_Y.min() >= 0:
        print("  ✅ REGRESIÓN GAMMA/LOG-NORMAL - Distribución sesgada positiva")
        
    if Variable_Y.nunique() == 2:
        print("  ✅ REGRESIÓN LOGÍSTICA - Variable binaria")
        
    if Variable_Y.min() >= 0 and all(Variable_Y % 1 == 0):
        print("  ✅ REGRESIÓN POISSON - Variable de conteo")
    
    return {
        'Es_Ordinal': Es_Ordinal,
        'Tiene_Muchos_Ceros': Tiene_Muchos_Ceros,
        'Es_Simetrica': Es_Simetrica,
        'Rango_Limitado': Rango_Limitado
    }